In [46]:
import mediapipe as mp
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cv2
from mpl_toolkits.mplot3d import Axes3D
import os


In [172]:
# extract keypoint
def extract_keypoint(path, normalize_extraction = True, real_extraction = True, label = True
                     save_path = None, show_video = False, save_file_name = None):
    mp_drawing = mp.solutions.drawing_utils
    mp_drawing_styles = mp.solutions.drawing_styles
    mp_pose = mp.solutions.pose
    
    landmark_names = [
        'nose',
        'left_eye_inner',
        'left_eye',
        'left_eye_outer',
        'right_eye_inner',
        'right_eye',
        'right_eye_outer',
        'left_ear',
        'right_ear',
        'mouth_left',
        'mouth_right',
        'left_shoulder',
        'right_shoulder',
        'left_elbow',
        'right_elbow',
        'left_wrist', 'right_wrist',
        'left_pinky_1', 'right_pinky_1',
        'left_index_1', 'right_index_1',
        'left_thumb_2', 'right_thumb_2',
        'left_hip', 'right_hip',
        'left_knee', 'right_knee',
        'left_ankle', 'right_ankle',
        'left_heel', 'right_heel',
        'left_foot_index', 'right_foot_index',
    ]
    landmark_list = []
    for i in landmark_names:
        for j in ["_x", "_y", "_z"]:
            landmark_list.append(i + j)
    
    
    results_normalize_points = []
    results_real_points = []
    
    cap = cv2.VideoCapture(path)
    with mp_pose.Pose(
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5) as pose:
        while cap.isOpened():
            success, image = cap.read()
            if not success:
                print("Done [%s] " % path)
                break

            image.flags.writeable = False
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image_height, image_width, _ = image.shape
            results = pose.process(image)
            
            
            #list생성
            if(normalize_extraction == True):
                normalize_points = []
                for i in range(len(landmark_names)):
                    try:
                        normalize_points.append(results.pose_world_landmarks.landmark[i].x)
                        normalize_points.append(results.pose_world_landmarks.landmark[i].y)
                        normalize_points.append(results.pose_world_landmarks.landmark[i].z)
                        
                    except:
                        normalize_points.append(None)
                        normalize_points.append(None)
                        normalize_points.append(None)    
                results_normalize_points.append(normalize_points)
                

            
            if(real_extraction == True):
                real_points = []
                for i in range(len(landmark_names)):
                    try:
                        real_points.append(results.pose_landmarks.landmark[i].x * image_width)
                        real_points.append(results.pose_landmarks.landmark[i].y * image_height)
                        real_points.append(results.pose_landmarks.landmark[i].z)
                        
                    except:
                        real_points.append(None)
                        real_points.append(None)
                        real_points.append(None)
                        
                results_real_points.append(real_points)
            
            if(show_video == True):
                image.flags.writeable = True
                image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
                mp_drawing.draw_landmarks(
                    image,
                    results.pose_landmarks,
                    mp_pose.POSE_CONNECTIONS,
                    landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())
                cv2.imshow('MediaPipe Pose', cv2.flip(image, 1))
                if cv2.waitKey(5) & 0xFF == 27:
                    break
                    
    cv2.destroyAllWindows()
    cap.release()
    if(save_path != None and save_file_name != None):
        #Normalize only
        if(normalize_extraction == True and real_extraction == False):
            point_normalize_dataframe = pd.DataFrame(columns = landmark_list, data = results_normalize_points)
            if not os.path.isdir(save_path):
                os.mkdir(save_path)
            point_normalize_dataframe.to_csv(save_path + save_file_name + "_noramlize.csv", index = False)
            return point_normalize_dataframe, _
        #real only
        elif(normalize_extraction == False and real_extraction == True):
            point_real_dataframe = pd.DataFrame(columns = landmark_list, data = results_real_points)
            if not os.path.isdir(save_path):
                os.mkdir(save_path)
            point_real_dataframe.to_csv(save_path + save_file_name + "_real.csv", index= False)
            return _, point_real_dataframe
    
        elif(normalize_extraction == True and real_extraction == True):
            point_normalize_dataframe = pd.DataFrame(columns = landmark_list, data = results_normalize_points)
            point_real_dataframe = pd.DataFrame(columns = landmark_list, data = results_real_points)
            if not os.path.isdir(save_path):
                os.mkdir(save_path)
            point_normalize_dataframe.to_csv(save_path + save_file_name + "_noramlize.csv", index = False)
            point_real_dataframe.to_csv(save_path + save_file_name + "_real.csv", index= False)
            return point_normalize_dataframe, point_real_dataframe


    
   

In [83]:
normalize, _ = extract_keypoint(path = "demo/demo.mp4", save_path = "demo/dataset/", show_video = True, save_file_name = "demo")

Done [demo/demo.mp4] 


In [58]:
demo = pd.read_csv("demo/demo_noramlize.csv")

In [169]:
#default start = 23 (hip, knee, ankle, heel, foot)
#
def compute_len_feature(dataframe = None, path = None, length = True,
                    drop_na = True, start = 23, end = 32, save_path = None, save_file_name = None):

    landmark_names = [
        'nose',
        'left_eye_inner',
        'left_eye',
        'left_eye_outer',
        'right_eye_inner',
        'right_eye',
        'right_eye_outer',
        'left_ear',
        'right_ear',
        'mouth_left',
        'mouth_right',
        'left_shoulder',
        'right_shoulder',
        'left_elbow',
        'right_elbow',
        'left_wrist', 'right_wrist',
        'left_pinky_1', 'right_pinky_1',
        'left_index_1', 'right_index_1',
        'left_thumb_2', 'right_thumb_2',
        'left_hip', 'right_hip',
        'left_knee', 'right_knee',
        'left_ankle', 'right_ankle',
        'left_heel', 'right_heel',
        'left_foot_index', 'right_foot_index',
    ]
    
    if(dataframe == None and path == None):
        return None
    
    else:
        if(dataframe == None):
            dataframe = pd.read_csv(path)
    if(drop_na == True):
        dataframe.dropna(axis = 0)
    
    result = pd.DataFrame()    
    landmark_list = list(dataframe.columns)
    
    if(length == True):
        for i in range(start, end, 2):
            x = dataframe[landmark_list[(i+1) * 3]] - dataframe[landmark_list[i * 3]]
            y = dataframe[landmark_list[(i+1) * 3 + 1]] - dataframe[landmark_list[i * 3 + 1]]
            point_len = np.sqrt(np.power(x, 2) + np.power(y, 2))
            point_depth = dataframe[landmark_list[(i+1) * 3 + 2]] - dataframe[landmark_list[i * 3 + 2]]
            
            string = landmark_names[i]
            string = string.replace('left', '')
            string = string.replace('right', '')
            string = string.replace('_', '')
            result[string + "_length"] = point_len
            result[string + "_depth"] =point_depth
            
    
    if(save_path != None and save_file_name != None):
        if not os.path.isdir(save_path):
            os.mkdir(save_path)
        result.to_csv(save_path + save_file_name + "_len_feauture.csv", index = False)
        return result

In [170]:
feature_len = compute_feature(path = "demo/dataset/demo_noramlize.csv", save_path = "demo/feature/", save_file_name = "demo")

In [171]:
feature_len.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 139 entries, 0 to 138
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   hip_length        139 non-null    float64
 1   hip_depth         139 non-null    float64
 2   knee_length       139 non-null    float64
 3   knee_depth        139 non-null    float64
 4   ankle_length      139 non-null    float64
 5   ankle_depth       139 non-null    float64
 6   heel_length       139 non-null    float64
 7   heel_depth        139 non-null    float64
 8   footindex_length  139 non-null    float64
 9   footindex_depth   139 non-null    float64
dtypes: float64(10)
memory usage: 11.0 KB


In [95]:
demo

,nose_x,nose_y,nose_z,left_eye_inner_x,left_eye_inner_y,left_eye_inner_z,left_eye_x,left_eye_y,left_eye_z,left_eye_outer_x,...,left_heel_z,right_heel_x,right_heel_y,right_heel_z,left_foot_index_x,left_foot_index_y,left_foot_index_z,right_foot_index_x,right_foot_index_y,right_foot_index_z
0,0.189573,-0.597068,-0.163225,0.164992,-0.641987,-0.153325,0.165396,-0.642102,-0.152595,0.165288,...,0.219943,-0.074937,0.806014,0.040866,0.054205,0.828724,0.167491,0.042717,0.835529,-0.055435
1,0.193588,-0.603334,-0.149474,0.167534,-0.646749,-0.137942,0.167917,-0.646882,-0.137253,0.167712,...,0.205442,-0.085021,0.802507,0.035435,0.042096,0.823012,0.151791,0.023676,0.831995,-0.059656
2,0.211302,-0.615537,-0.139884,0.186049,-0.658645,-0.128379,0.186389,-0.658804,-0.127688,0.186094,...,0.213280,-0.098092,0.792451,0.047457,0.034728,0.805992,0.166923,0.011385,0.824457,-0.049063
3,0.217025,-0.623345,-0.139500,0.192396,-0.666512,-0.128613,0.192676,-0.666626,-0.127903,0.192430,...,0.213796,-0.129218,0.782278,0.070536,0.084782,0.796275,0.167291,-0.022285,0.816900,-0.034294
4,0.217587,-0.621558,-0.133263,0.193579,-0.664917,-0.123402,0.193843,-0.665022,-0.122657,0.193650,...,0.213007,-0.149983,0.771653,0.102458,0.119917,0.781949,0.166544,-0.048812,0.805134,0.001875
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134,0.172675,-0.603860,-0.098230,0.157629,-0.645690,-0.085949,0.157888,-0.645668,-0.085247,0.157707,...,0.252255,-0.020252,0.797077,0.084293,0.047086,0.842310,0.200128,0.080760,0.829856,0.010952
135,0.172800,-0.604111,-0.100596,0.157548,-0.645890,-0.088546,0.157805,-0.645877,-0.087849,0.157620,...,0.253856,-0.019843,0.797681,0.083454,0.046662,0.837415,0.203124,0.081989,0.829422,0.009869
136,0.172851,-0.603654,-0.103807,0.157453,-0.645369,-0.092017,0.157710,-0.645374,-0.091318,0.157522,...,0.252636,-0.020226,0.800890,0.083016,0.044516,0.837369,0.202032,0.082571,0.832004,0.007272
137,0.172274,-0.601551,-0.108398,0.156698,-0.643332,-0.096937,0.156964,-0.643359,-0.096241,0.156786,...,0.249989,-0.020457,0.802043,0.076922,0.044526,0.836991,0.199022,0.083109,0.832080,-0.003527


In [93]:
p0 = [5, 0]
p1 = [0, 0] # 중점
p2 = [0, 5]

In [113]:
import numpy as np

p0 = [3.5, 6.7]
p1 = [7.9, 8.4]
p2 = [10.8, 4.8]

'''
compute angle (in degrees) for p0p1p2 corner
Inputs:
    p0,p1,p2 - points in the form of [x,y]
'''

v0 = np.array(p0) - np.array(p1)
v1 = np.array(p2) - np.array(p1)

angle = np.math.atan2(np.linalg.det([v0,v1]),np.dot(v0,v1))
print(np.degrees(angle))

107.72865519428085


In [161]:
p0 = np.array([demo["nose_x"].tolist(), demo["nose_y"].tolist()])
p1 = np.array([demo["left_eye_x"].tolist(), demo["left_eye_y"].tolist()])
p2 = np.array([demo["left_eye_x"].tolist(), demo["left_eye_y"].tolist()])

In [167]:
for i in range(len(p0[0])):
    v0 = np.array(p0[:, i]) - np.array(p1[:, i])
    v1 = np.array(p2[:, i]) - np.array(p1[:, i])
    print(v0)
    print(v1)
    angle = np.math.atan2(np.linalg.det([v0,v1]),np.dot(v0,v1))
    print(np.degrees(angle))


[0.02417742 0.04503351]
[0. 0.]
0.0
[0.02567138 0.04354745]
[0. 0.]
0.0
[0.02491324 0.04326755]
[0. 0.]
0.0
[0.02434899 0.0432806 ]
[0. 0.]
0.0
[0.02374367 0.0434643 ]
[0. 0.]
0.0
[0.01925828 0.04314798]
[0. 0.]
0.0
[0.01649873 0.04253352]
[0. 0.]
0.0
[0.01620473 0.04379922]
[0. 0.]
0.0
[0.01696171 0.04320353]
[0. 0.]
0.0
[0.01665686 0.0428564 ]
[0. 0.]
0.0
[0.01674736 0.04242337]
[0. 0.]
0.0
[0.01705387 0.04222214]
[0. 0.]
0.0
[0.01717779 0.04201269]
[0. 0.]
0.0
[0.01902261 0.04193127]
[0. 0.]
0.0
[0.01899567 0.04221189]
[0. 0.]
0.0
[0.01652971 0.04362118]
[0. 0.]
0.0
[0.01338699 0.04426354]
[0. 0.]
0.0
[0.01326443 0.04459602]
[0. 0.]
0.0
[0.01309989 0.04509699]
[0. 0.]
0.0
[0.01592614 0.04497313]
[0. 0.]
0.0
[0.01827657 0.04434466]
[0. 0.]
0.0
[0.01977244 0.04419726]
[0. 0.]
0.0
[0.02144225 0.04341668]
[0. 0.]
0.0
[0.02399111 0.04321879]
[0. 0.]
0.0
[0.02563864 0.04270905]
[0. 0.]
0.0
[0.02736059 0.04208946]
[0. 0.]
0.0
[0.02791461 0.04210395]
[0. 0.]
0.0
[0.02602585 0.04221153]
[0. 

In [153]:
test = np.array(p0)

In [162]:
test

array([[ 0.18957308,  0.19358793,  0.211302  ,  0.21702486,  0.21758716,
         0.23076257,  0.23856084,  0.24191979,  0.25281015,  0.26391309,
         0.2707161 ,  0.28417844,  0.28600425,  0.28504398,  0.28080952,
         0.27226707,  0.26020142,  0.2607801 ,  0.26036128,  0.26387236,
         0.26251322,  0.25427487,  0.2529206 ,  0.24700789,  0.24404895,
         0.24152465,  0.23777725,  0.2213107 ,  0.209383  ,  0.190567  ,
         0.19046019,  0.19540848,  0.19827376,  0.19963375,  0.20153144,
         0.20219675,  0.21176608,  0.22027482,  0.22193407,  0.21837249,
         0.21422501,  0.20617488,  0.20115684,  0.19902031,  0.19412072,
         0.19456233,  0.19942775,  0.2006153 ,  0.20114726,  0.21169335,
         0.22651605,  0.22764163,  0.2236927 ,  0.22203706,  0.210722  ,
         0.19580649,  0.18727353,  0.18919292,  0.19315982,  0.19410676,
         0.19071311,  0.19108671,  0.19362965,  0.19833384,  0.19705828,
         0.18715334,  0.17809211,  0.1776855 ,  0.1

In [160]:
test[:, 0]

array([ 0.18957308, -0.59706831])

In [158]:
len(test[:, ])

2

In [154]:
test.shape

(2, 139)

In [ ]:
test.reshape(len,2)

In [138]:
p0[0].tolist()[0]

0.1895730793476104